# Automated script to download images from a Google Earth Engine using GeeMap

## Import Libraries and Packages

In [1]:
!pip install geemap
!pip install earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.9 MB/s eta 0:00:00


In [6]:
import ee
import geemap
import ee
import webbrowser
import ipywidgets as widgets

## Autentication in Google Colab and GEE


In [3]:
auth_url = ee.Authenticate(auth_mode='notebook')
webbrowser.open(auth_url)
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/drive%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=7sQRQ2M0pkceeqGhRGPrroXbZ4-CpjjncB3Fdl27wkA&tc=hIhU2Dozkhtwu18jsKqOhD2kBJ1nTmzPyhDDHoRM_1c&cc=pJulEfYveMOSAfg8N6pZViuUnP7Mn6WumTTkUygXxDc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ASc3gC0YzNOOASeMtM0Faam-tDhbTfOVy4H3cUSlAwrNErmfqFVyznwN0sU

Successfully saved authorization token.


## Import datasets to search countries, departments and cities

In [25]:
# Cargar dataset geoBoundaries ADM2 (versión WM/geoLab)
geoboundaries = ee.FeatureCollection("WM/geoLab/geoBoundaries/600/ADM2")

# Crear mapa
Map = geemap.Map(center=[0, 0], zoom=2)

# Cuadro de texto para insertar ciudad/municipio
text_input = widgets.Text(
    value='Cobija',
    description='Buscar:',
    placeholder='Escribe nombre exacto'
)

# Panel de salida para metadatos
metadata_output = widgets.Output()

# Función para buscar y mostrar resultados usando solo shapeName
def search_boundary(change):
    keyword = change['new']
    results = geoboundaries.filter(
        ee.Filter.stringContains("shapeName", keyword)
    )
    Map.clear_layers()
    Map.addLayer(results, {"color":"blue"}, f"Resultados: {keyword}")
    Map.centerObject(results)

    # Mostrar metadatos
    metadata_output.clear_output()
    features = results.limit(5).getInfo().get('features', [])
    with metadata_output:
        if features:
            for f in features:
                props = f['properties']
                print("=== Metadatos ===")
                print(f"Nombre: {props.get('shapeName')}")
                print(f"País: {props.get('shapeGroup')}")
                print(f"Nivel ADM: {props.get('shapeType')}")
                print(f"ISO Código: {props.get('shapeISO')}")
                print("------------------")
        else:
            print("No se encontraron resultados para:", keyword)

# Conectar cuadro de texto con función
text_input.observe(search_boundary, names='value')

# Mostrar cuadro de texto, mapa y panel de metadatos
display(text_input)
display(metadata_output)
Map

Text(value='Cobija', description='Buscar:', placeholder='Escribe nombre exacto')

Output()

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

TypeError: 'NoneType' object is not subscriptable

TypeError: 'NoneType' object is not subscriptable

## Create interactive basemaps

In [13]:
Map = geemap.Map(center=[4.6, -74.1], zoom=6)

# Definir 10 basemaps de la lista oficial
styles = {
    "OpenStreetMap": "OpenStreetMap",
    "Esri World Street Map": "Esri.WorldStreetMap",
    "Esri World Imagery": "Esri.WorldImagery",
    "Esri World Topo Map": "Esri.WorldTopoMap",
    "Esri NatGeo World Map": "Esri.NatGeoWorldMap",
    "Esri Ocean Basemap": "Esri.OceanBasemap",
    "Esri World Gray Canvas": "Esri.WorldGrayCanvas",
    "Esri World Physical": "Esri.WorldPhysical",
    "Esri World Shaded Relief": "Esri.WorldShadedRelief",
    "Esri World Terrain": "Esri.WorldTerrain",
    "CartoDB DarkMatter": "CartoDB.DarkMatter",
    "CartoDB Positron": "CartoDB.Positron",
    "CartoDB Voyager": "CartoDB.Voyager",
    "OpenTopoMap": "OpenTopoMap"
}

# Crear menú desplegable
style_dropdown = widgets.Dropdown(
    options=list(styles.keys()),
    value="OpenStreetMap",
    description="Basemap:"
)

# Función para actualizar basemap
def update_style(change):
    Map.clear_layers()  # Limpia capas previas
    Map.add_basemap(styles[change['new']])  # Añade el basemap seleccionado

# Conectar menú con función
style_dropdown.observe(update_style, names='value')

# Mostrar menú y mapa
display(style_dropdown)
Map


Dropdown(description='Basemap:', options=('OpenStreetMap', 'Esri World Street Map', 'Esri World Imagery', 'Esr…

Map(center=[4.6, -74.1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

## Create prompt to search datasets in GEE related with water resources and hydrology